# Problem 3 and Challenge Problem

1. Find out the best data-structure to represent / store the graph in memory.
2. Devise an algorithm to assign the labels to the vertices using vertex **k**-labeling definition. (**Main Task**)
3. How traversing will be applied?
4. Store the labels of vertices and weights of the edges as an outcome.
5. Compare your results with mathematical property and tabulate the outcomes for comparison.


In [ ]:
class Edge:
  # Data Fields
  source = None
  dest = None
  weight = None

  # Constructor
  def __init__(self, source, dest, weight=1.0):
    self.source = source
    self.dest = dest
    self.weight = weight

  # Destructor
  def __del__(self):
    self.source = None
    self.dest = None
    self.weight = None

  # Member Functions
  # Compare edges for equality "=="
  def __eq__(self, other):
    # Check if 'other' is an instance of the same class
    if not isinstance(other, Edge):
      return False
    return self.source == other.source and self.dest == other.dest

  # Returns the destination vertex
  def get_dest(self):
    return self.dest

  # Returns te source vertex
  def get_source(self):
    return self.source

  # Returns the weight
  def get_weight(self):
    return self.weight

  # Change the weight of an edge
  def set_weight(self, weight):
    self.weight = weight

  # Returns a string representation of the edge "str()"
  def __str__(self):
    return f"({self.source}, {self.dest}, {self.weight})"

In [ ]:
from collections import deque
import math

class Graph:
  # Data fields
  directed = False
  num_v = 0
  graph = {}
  edges = {}
  labels = {}

  # Constructor
  def __init__(self, num_v=0, directed=False):
    self.directed = directed
    self.num_v = num_v

  # Destructor
  def __del__(self):
    self.directed = None
    self.num_v = None
    self.graph = None
    self.edges = None
    self.labels = None

  # Member functions
  # Returns the number of vertices
  def get_num_v(self):
    return self.num_v

  # Returns true if the graph is directed
  def is_directed(self):
    return self.directed

  # Returns the list of vertices adjacent to a given source
  def begin(self, source):
    # Using .get for safety, returning empty list if source has no outgoing edges
    return self.graph.get(source, [])

  # Gets the edge between 2 vertices
  def get_edge(self, source, dest):
    if self.is_edge(source, dest):
      for edge in self.edges[source]:
        if edge.get_dest() == dest:
          return edge
    return None

  # Inesrts a new edge into the "edges" dictionary
  def insert_edge(self, edge):
    src, dest, weight = edge.get_source(), edge.get_dest(), edge.get_weight()
    # Base case: Exit function if the edge already exists
    if self.is_edge(src, dest):
      return

    # Inserts the edge into the "edges" dictionary
    if src in self.edges:
      self.edges[src].append(edge)
    else:
      self.edges[src] = [edge]

    # Inserts the edge with the source and dest swapped if the graph is not directed
    if not self.directed:
      opp_edge = Edge(dest, src, weight)
      # Prevent infinite recursion if edge already exists for the opposite direction
      if not self.is_edge(dest, src):
        self.insert_edge(opp_edge)

    # Insert the vertices into the "graph" dictionary
    self.insert_graph(edge)
    if not self.directed:
      opp_edge = Edge(dest, src, weight)
      self.insert_graph(opp_edge)

  # Inserts the vertices from 'edge' into the "graph" dictionary
  def insert_graph(self, edge):
    src, dest, weight = edge.get_source(), edge.get_dest(), edge.get_weight()
    # Inserts the vertices ordered by smallest weight to largest weight
    if src in self.graph:
      # Prevents duplicate insertions
      if dest in self.graph[src]:
        return
      for i in self.graph[src]:
        e = self.get_edge(src, i)
        if e and e.get_weight() > weight:
          index = self.graph[src].index(i)
          self.graph[src].insert(index, dest)
          return    # ensures no append at the end of list
      # Only reached if no insertion happened earlier
      self.graph[src].append(dest)
    else:
      self.graph[src] = [dest]

  # Determines whether an edge exists from source to dest
  def is_edge(self, source, dest):
    if source in self.edges:
      for i in self.edges[source]:
        if i.get_dest() == dest:
          return True
    return False

  # Simple BFS (change to account for weight and source labels)
  #     BFS = Breadth First Search
  def bfs(self, start):
    seen = set([start])
    q = deque([start])
    order = []
    while q:
        v = q.popleft()
        order.append(v)
        if v in self.graph: # Check if v has neighbors
            for nbr in self.graph[v]:
                if nbr not in seen:
                    seen.add(nbr)
                    q.append(nbr)
    return order

  # Simple DFS (change to account for weight and source labels)
  #     DFS = Depth First Search
  def dfs(self, start):
    seen = set()
    order = []
    def rec(v):
        seen.add(v)
        order.append(v)
        if v in self.graph: # Check if v has neighbors
            for nbr in self.graph[v]:
                if nbr not in seen:
                    rec(nbr)
    rec(start)
    return order

  def print_edge(self, edge):
    src, dest, weight = edge.get_source(), edge.get_dest(), edge.get_weight()
    # Use labels if available, otherwise use vertex ID
    src_label = self.labels.get(src, src)
    dest_label = self.labels.get(dest, dest)
    return f"({src_label}, {dest_label}, {weight})"

  # Print the ternary tree
  def print_tree(self):
    if not self.labels:
      print("No vertex labels available.")
      return

    # BFS Traversal to print level by level
    from collections import deque
    visited = set()
    queue = deque([(0, 0)])   # (vertex, level)

    current_level = 0
    line = []

    while queue:
      vertex, level = queue.popleft()
      if vertex in visited:
        continue
      visited.add(vertex)
    # If at new level, print the previous line
      if level != current_level:
        print("Level", current_level, ":", " ".join(map(str, line)))
        line = []
        current_level = level

      # Add the label instead of the vertex ID, using .get for safety
      line.append(self.labels.get(vertex, vertex))

      # Enqueue children
      if vertex in self.graph: # Check if vertex has neighbors
        for child in self.graph[vertex]:
          queue.append((child, level + 1))

    # Print the last line
    if line:
      print("Level", current_level, ":", " ".join(map(str, line)))

  # Generate a perfect ternary tree
  def generate_m_ary_tree(self, m=3, h=5):
    '''
    Generate a perfect m-ary tree.
    m: branching factor (degree)
    h: height (levels + 1)
    '''
    # Total number of nodes in perfect m-ary tree
    self.num_v = ((m ** (h - 1)) - 1) // (m - 1)

    # Build tree level by level
    current_parent = 0
    next_child = 1

    # For each node, assign m children until we reach num_v
    while next_child < self.num_v:
      for _ in range(m):
        if next_child >= self.num_v:
          break
        edge = Edge(current_parent, next_child)
        self.insert_edge(edge)
        next_child += 1
      current_parent += 1

    k_internal = math.ceil(self.num_v/2)
    self.assign_k_labels(k_internal, m, h)

  def assign_k_labels(self, k, m=3, h=5):
    '''
    Generate a k-labeling for the graph.
    k: max value of labels
    '''
    # Traverse through the tree using DFS and assign labels along the order
    dfs = self.dfs(0)
    # Use BFS to track levels and parents
    bfs = self.bfs(0)

    # Create a Dictionary to store the nodes at each height
    levels = {key: [] for key in range(h-1)}
    height = 0
    first = 0
    last = 1
    # Seperate the BFS order into levels
    for i in bfs:
      if bfs.index(i) == last:
        first = last
        last = (first * m) + 1
        height += 1
      levels[height].append(i)

    parent_map = {}
    # Assign child with parent (skip the root)
    for lvl in range(1, h - 1):
      parents = levels[lvl - 1]
      children = levels[lvl]

      # Each parent has m children
      for i, child in enumerate(children):
        parent_index = i // m
        parent = parents[parent_index]
        parent_map[child] = parent

    # Assign label to root and second node (they are fixed)
    self.labels[0] = 1
    self.labels[1] = 1
    edge = self.get_edge(0, 1)
    edge.set_weight(2)
    if not self.directed:
      opp_edge = self.get_edge(1, 0)
      opp_edge.set_weight(2)


    # Assign the labels for the root children
    root_child = []
    for i in range(1, m):
      root_child.append(math.ceil((i*k)/(m-1)))
      self.labels[levels[1][i]] = root_child[i - 1]
      edge = self.get_edge(0, i + 1)
      edge.set_weight(root_child[i - 1] + 1)
      if not self.directed:
        opp_edge = self.get_edge(i + 1, 0)
        opp_edge.set_weight(root_child[i - 1] + 1)

    # Assign labels (skipping first 2)
    weight = 3
    for i in range(2, (m ** (h - 2)) + 1):
      child = dfs[i]
      parent = parent_map[child]

      if child in range(1, m + 1):
        continue

      label = weight - self.labels[parent]

      # Skip the weights of the root children
      for i in root_child:
        if weight == i + 1:
          weight += 1
          label = weight - self.labels[parent]

      self.labels[child] = label
      edge = self.get_edge(parent, child)
      edge.set_weight(weight)
      if not self.directed:
        opp_edge = self.get_edge(child, parent)
        opp_edge.set_weight(weight)
      weight += 1

    # Assign last branch in reverse
    weight = self.num_v
    for i in range(m ** (h - 2) + 1, self.num_v):
      child = dfs[i]
      parent = parent_map[child]
      label = weight - self.labels[parent]
      self.labels[child] = weight - self.labels[parent]
      edge = self.get_edge(parent, child)
      edge.set_weight(weight)
      if not self.directed:
        opp_edge = self.get_edge(child, parent)
        opp_edge.set_weight(weight)
      weight -= 1

In [ ]:
# Create a graph
new_graph = Graph()
new_graph.generate_m_ary_tree()
new_graph.print_tree()

Level 0 : 1
Level 1 : 1 10 20
Level 2 : 2 6 11 6 10 15 20 16 12
Level 3 : 2 3 4 2 3 4 2 3 4 11 12 13 12 13 14 11 12 13 19 18 17 19 18 17 19 18 17


In [ ]:
# Traversals
print("BFS: ", new_graph.bfs(0))
print("DFS: ", new_graph.dfs(0))

BFS:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
DFS:  [0, 1, 4, 13, 14, 15, 5, 16, 17, 18, 6, 19, 20, 21, 2, 7, 22, 23, 24, 8, 25, 26, 27, 9, 28, 29, 30, 3, 10, 31, 32, 33, 11, 34, 35, 36, 12, 37, 38, 39]


In [ ]:
# Build adjacency matrix
# Determine the maximum vertex ID to correctly size the matrix
max_vertex_id = -1

# Consider all vertex IDs from edges
if new_graph.edges:
    for src in new_graph.edges:
        max_vertex_id = max(max_vertex_id, src)
        for e in new_graph.edges[src]:
            max_vertex_id = max(max_vertex_id, e.get_dest())

# Also consider all vertex IDs from labels, as vertices might exist without being part of an edge
if new_graph.labels:
    for v_id in new_graph.labels:
        max_vertex_id = max(max_vertex_id, v_id)

# If graph is empty (no edges, no labels), fall back to new_graph.get_num_v() or default to 0
if max_vertex_id == -1:
    # Use the initial num_v if no edges/labels were found, or ensure at least 1 for a 1x1 matrix
    n = new_graph.get_num_v()
    if n == 0:
        n = 1
else:
    n = max_vertex_id + 1

matrix = [[0]*n for _ in range(n)]
for src in new_graph.edges:
    for e in new_graph.edges[src]:
        matrix[e.get_source()][e.get_dest()] = e.get_weight()

In [ ]:
# Comparison table
print("\nComparison Table:")
print("{:<8} {:<15} {:<20} {:<40}".format(
    "Vertex", "Label", "Adjacency List", "Edge List"))
print("-"*90)

for v in range(new_graph.get_num_v()):
    label = new_graph.labels.get(v, "")
    adj_list = new_graph.graph.get(v, [])
    edge_list = [new_graph.print_edge(e) for e in new_graph.edges.get(v, [])]

    print("{:<8} {:<15} {:<20} {:<40}".format(
        v, str(label), str(adj_list), str(edge_list)
    ))





Comparison Table:
Vertex   Label           Adjacency List       Edge List                               
------------------------------------------------------------------------------------------
0        1               [1, 2, 3]            ['(1, 1, 2)', '(1, 10, 11)', '(1, 20, 21)']
1        1               [0, 4, 5, 6]         ['(1, 1, 2)', '(1, 2, 3)', '(1, 6, 7)', '(1, 11, 12)']
2        10              [0, 7, 8, 9]         ['(10, 1, 11)', '(10, 6, 16)', '(10, 10, 20)', '(10, 15, 25)']
3        20              [0, 10, 11, 12]      ['(20, 1, 21)', '(20, 20, 40)', '(20, 16, 36)', '(20, 12, 32)']
4        2               [1, 13, 14, 15]      ['(2, 1, 3)', '(2, 2, 4)', '(2, 3, 5)', '(2, 4, 6)']
5        6               [1, 16, 17, 18]      ['(6, 1, 7)', '(6, 2, 8)', '(6, 3, 9)', '(6, 4, 10)']
6        11              [1, 19, 20, 21]      ['(11, 1, 12)', '(11, 2, 13)', '(11, 3, 14)', '(11, 4, 15)']
7        6               [2, 22, 23, 24]      ['(6, 10, 16)', '(6, 11, 17)', '(6, 12, 1

In [ ]:
# Perfect Ternary Tree statistics
all_weights = []
for src in new_graph.edges:
    for e in new_graph.edges[src]:
        all_weights.append(e.get_weight())

E = len(all_weights)
min_weight = min(all_weights) if all_weights else None
max_weight = max(all_weights) if all_weights else None

print("\nGraph Statistics:")
print(f"Total edges (E): {math.ceil(E/2)}")  # Divided by 2 because the graph is undirected
print(f"Minimum weight: {min_weight}")
print(f"Maximum weight: {max_weight}")



Graph Statistics:
Total edges (E): 39
Minimum weight: 2
Maximum weight: 40
